In [ ]:
import pandas as pd

# Path to the original CSV file
csv_file_path = '/content/Issues_Collected_category and issue - Issues consolidatd (1).csv'

# Read the original CSV file
df = pd.read_csv(csv_file_path)

# Calculate the number of unique categories and issue types
num_categories = len(df['Category'].unique())
num_subcategories = len(df['Issue type'].unique())

print("Number of categories:", num_categories)
print("Number of sub-categories:", num_subcategories)


In [ ]:
df.head()

In [ ]:
# Print all unique issue types
unique_issue_types = df["Issue type"].unique()
print("Unique Issue Types:")
print(unique_issue_types)

# Print the count of each issue type
issue_type_counts = df["Issue type"].value_counts()
print("Issue Type Counts:")
print(issue_type_counts)

In [ ]:
import pickle

import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
import cv2
import numpy as np

# Load labels
df = pd.read_csv('/content/Issues_Collected_category and issue - Issues consolidatd (1).csv')
labels = df['Issue type'].values
photos = df['Photo'].values

# Convert labels to numerical values
le = LabelEncoder()
labels = le.fit_transform(labels)

with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

# Preprocess images
image_list = []
for photo in photos:
    image = load_img('/content/drive/MyDrive/Photos_Walking Audit/' + photo, target_size=(299, 299))
    image = img_to_array(image)
    image = preprocess_input(image)
    image_list.append(image)
images = np.array(image_list)

# Define the model
base_model = Xception(include_top=False, weights='imagenet', input_shape=(299, 299, 3), pooling='avg')
output = Dense(len(le.classes_), activation='softmax')(base_model.output)
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(images, labels, epochs=10, validation_split=0.2)

# Save the model
model.save('xception_model.h5')
